<a href="https://colab.research.google.com/github/nlquantumm-source/SFQ/blob/main/SFQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# @title Block 1: Imports and System Configuration
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from ipywidgets import interact, interactive, FloatSlider, IntSlider
from IPython.display import display

# [CRITICAL FIX] Initialize Colab Renderer
pio.renderers.default = 'colab'

def normalize_state(psi):
    """
    Normalizes the quantum state vector.

    Args:
        psi (np.ndarray): The complex state vector.

    Returns:
        np.ndarray: Normalized state vector.
    """
    norm = np.linalg.norm(psi)
    if norm == 0:
        return psi
    return psi / norm

def get_bloch_vector(psi):
    """
    Converts Qutrit state to 3D Bloch vector + Leakage Color.

    Args:
        psi (np.ndarray): 3-element complex state vector [c0, c1, c2].

    Returns:
        tuple: (x, y, z, leakage_prob)
    """
    # Projection onto Qubit Subspace
    x = 2 * np.real(psi[0] * np.conj(psi[1]))
    y = 2 * np.imag(psi[0] * np.conj(psi[1]))
    z = np.abs(psi[0])**2 - np.abs(psi[1])**2

    # Leakage probability (State |2>)
    leakage = np.abs(psi[2])**2
    return x, y, z, leakage

print("Block 1: Imports and Helper Functions loaded successfully.")

Block 1: Imports and Helper Functions loaded successfully.


In [31]:
# @title Block 2: The State Rotation Matrix Engine
def sfq_rotation_matrix(delta_theta, lam):
    """
    Constructs the 3x3 SFQ State Rotation Matrix based on the screenshot.

    This matrix models the evolution of the system |0>, |1>, |2> after
    a single 'Digital Kick'.

    Args:
        delta_theta (float): The 'Kick Angle' per pulse (in radians).
        lam (float): The Lambda parameter (Leakage Coupling Strength).

    Returns:
        np.ndarray: A 3x3 Unitary Matrix.
    """
    # Kappa normalization
    kappa = np.sqrt(1 + lam**2)
    kappa_sq = kappa**2

    # Angles
    arg_2 = (kappa * delta_theta) / 2
    arg_4 = (kappa * delta_theta) / 4

    c_2 = np.cos(arg_2)
    s_2 = np.sin(arg_2)
    s_4 = np.sin(arg_4)

    # Matrix Elements (Exact transcription)
    u00 = lam**2 + c_2
    u01 = -kappa * s_2
    u02 = 2 * lam * (s_4**2)

    u10 = kappa * s_2
    u11 = kappa_sq * c_2
    u12 = -kappa * lam * s_2

    u20 = 2 * lam * (s_4**2)
    u21 = kappa * lam * s_2
    u22 = 1 + (lam**2 * c_2)

    # Combine into matrix and apply 1/kappa^2 pre-factor
    U_sfq = (1 / kappa_sq) * np.array([
        [u00, u01, u02],
        [u10, u11, u12],
        [u20, u21, u22]
    ], dtype=complex)

    return U_sfq

print("Block 2: Rotation Matrix Engine initialized.")

Block 2: Rotation Matrix Engine initialized.


In [36]:
# @title Block 3: 3D Video Simulation Generator (Dual View)
def run_simulation(kick_angle, leakage_lambda, steps):
    """
    Generates the animation frames by simulating a sequence of SFQ pulses.
    Visualizes both the Bloch Vector and the Cumulative Rotation Matrix.
    """
    # Initial State |0>
    psi = np.array([1.0, 0.0, 0.0], dtype=complex)
    # Initial Cumulative Matrix (Identity)
    U_cum = np.eye(3, dtype=complex)

    # Single Step Rotation
    U_step = sfq_rotation_matrix(kick_angle, leakage_lambda)

    # History Data
    xs, ys, zs, leaks = [], [], [], []
    matrices = []

    # Generate Trajectory
    for _ in range(steps):
        # 1. Record State
        x, y, z, l = get_bloch_vector(psi)
        xs.append(x); ys.append(y); zs.append(z); leaks.append(l)

        # 2. Record Matrix (Magnitude)
        matrices.append(np.abs(U_cum))

        # 3. Evolve
        psi = np.dot(U_step, psi)
        psi = normalize_state(psi)
        U_cum = np.dot(U_step, U_cum)

    # --- Build Plotly Animation (Dual Subplots) ---
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'heatmap'}]],
        subplot_titles=("Bloch Sphere (State)", "Rotation Matrix Magnitude |U^n|"),
        column_widths=[0.6, 0.4]
    )

    # 1. Background Sphere (Wireframe) for Scene 1
    phi = np.linspace(0, 2*np.pi, 30)
    theta = np.linspace(0, np.pi, 30)
    sphere_x = np.outer(np.cos(phi), np.sin(theta))
    sphere_y = np.outer(np.sin(phi), np.sin(theta))
    sphere_z = np.outer(np.ones(30), np.cos(theta))

    # Trace 0: Static Sphere (We will NOT update this in frames)
    fig.add_trace(go.Surface(x=sphere_x, y=sphere_y, z=sphere_z, opacity=0.1, showscale=False, colorscale='Greys'), row=1, col=1)

    # 2. Initial Traces
    # Trace 1: Vector
    fig.add_trace(go.Scatter3d(x=[0, 1], y=[0, 0], z=[0, 0], mode='lines', line=dict(color='black', width=5), name='State'), row=1, col=1)
    # Trace 2: Trail
    fig.add_trace(go.Scatter3d(x=[], y=[], z=[], mode='lines', line=dict(color='blue'), name='Trail'), row=1, col=1)
    # Trace 3: Tip
    fig.add_trace(go.Scatter3d(x=[1], y=[0], z=[0], mode='markers', marker=dict(size=5, color='red'), name='Tip'), row=1, col=1)
    # Trace 4: Heatmap
    fig.add_trace(go.Heatmap(
        z=np.eye(3),
        x=['|0>', '|1>', '|2>'],
        y=['<0|', '<1|', '<2|'],
        colorscale='Viridis', zmin=0, zmax=1,
        showscale=True,
        colorbar=dict(title="|Amp|", x=1.05)
    ), row=1, col=2)

    # 3. Frames
    frames = []
    for k in range(len(xs)):
        frames.append(go.Frame(
            data=[
                # Optimize: Do NOT include the static sphere (Trace 0) here.
                go.Scatter3d(x=[0, xs[k]], y=[0, ys[k]], z=[0, zs[k]]), # Update Trace 1
                go.Scatter3d(x=xs[:k+1], y=ys[:k+1], z=zs[:k+1]),       # Update Trace 2
                go.Scatter3d(x=[xs[k]], y=[ys[k]], z=[zs[k]], marker=dict(color=leaks[k], colorscale='Hot', cmin=0, cmax=0.5)), # Update Trace 3
                go.Heatmap(z=matrices[k])                               # Update Trace 4
            ],
            name=f'fr{k}',
            traces=[1, 2, 3, 4] # Only update dynamic traces
        ))

    fig.frames = frames

    # 4. Layout
    fig.update_layout(
        title=f"SFQ Simulation: Angle={kick_angle:.2f}, Leakage={leakage_lambda:.2f}",
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='cube'),
        height=600,
        updatemenus=[{
            'type': 'buttons',
            'showactive': False,
            'buttons': [{
                'label': 'Play',
                'method': 'animate',
                'args': [None, {'frame': {'duration': 100, 'redraw': True}, 'fromcurrent': True}]
            }, {
                'label': 'Pause',
                'method': 'animate',
                'args': [[], {'mode': 'immediate', 'frame': {'duration': 0, 'redraw': False}}]
            }]
        }]
    )

    # [CRITICAL FIX] Force 'colab' renderer to ensures the plot appears in the output cell
    fig.show(renderer='colab')

In [35]:
# @title Block 4: Control Knobs (Interactive Execution)

print("1. Generating initial Dual-View Animation...\n")
run_simulation(kick_angle=0.15, leakage_lambda=0.2, steps=50)

print("\n2. Loading Interactive Controls...\n")
w = interactive(run_simulation,
         kick_angle=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.15, description='Kick Angle (rad)', continuous_update=False),
         leakage_lambda=FloatSlider(min=0.0, max=0.5, step=0.05, value=0.2, description='Leakage (λ)', continuous_update=False),
         steps=IntSlider(min=20, max=150, step=10, value=50, description='# of Pulses', continuous_update=False))
display(w)

1. Generating initial Dual-View Animation...




2. Loading Interactive Controls...



interactive(children=(FloatSlider(value=0.15, continuous_update=False, description='Kick Angle (rad)', max=1.0…

In [54]:
# @title Block 5a: Install Video Export Dependencies
# 1. Install Python Libraries
!pip install -q -U plotly>=6.1.1 kaleido

# 2. Install Google Chrome Stable
# We use the official .deb to avoid Snap package issues in Colab (Ubuntu 22.04)
print("Downloading and installing Google Chrome... (This takes about 1 minute)")
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get update -qq
!apt-get install -y ./google-chrome-stable_current_amd64.deb > /dev/null

print("Google Chrome installed successfully. Please run Block 5b.")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Google Chrome installed successfully. Please run Block 5b.


In [55]:
# @title Block 5b: Generate and Download MP4 Video
import os
import sys
import numpy as np
from google.colab import files

# --- HOTFIX: Setup Browser Path for Kaleido ---
# Prefer Google Chrome (stable) over Chromium
if os.path.exists("/usr/bin/google-chrome"):
    os.environ["KALEIDO_CHROMIUM_PATH"] = "/usr/bin/google-chrome"
elif os.path.exists("/usr/bin/chromium-browser"):
    os.environ["KALEIDO_CHROMIUM_PATH"] = "/usr/bin/chromium-browser"

# --- HOTFIX: Hard Reset of Plotly Library ---
# We continue to purge the cache to ensure the fresh environment is used.
print("Checking environment for video export...")

# 1. Check Kaleido
try:
    import kaleido
    # Try to access version to ensure it's loaded
    k_ver = getattr(kaleido, '__version__', 'unknown')
    print(f"Kaleido is installed (version {k_ver}).")
except ImportError:
    raise ImportError("Kaleido not found. Please run Block 5a.")

# 2. Purge Plotly Cache
print("Performing hard reset of Plotly modules...")
modules_to_purge = [k for k in sys.modules if k.startswith('plotly') or k.startswith('_plotly_utils') or k.startswith('kaleido')]
for k in modules_to_purge:
    try:
        del sys.modules[k]
    except KeyError:
        pass
print(f"Successfully purged {len(modules_to_purge)} modules.")

# 3. Re-import Plotly (Fresh Load from Disk)
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

def export_sfq_video(kick_angle, leakage_lambda, steps, filename="sfq_video.mp4"):
    print(f"1. Initializing Simulation (Angle={kick_angle}, Lambda={leakage_lambda})...")

    # --- 1. Re-run Simulation Logic to get Data ---
    psi = np.array([1.0, 0.0, 0.0], dtype=complex)
    U_cum = np.eye(3, dtype=complex)
    U_step = sfq_rotation_matrix(kick_angle, leakage_lambda)

    xs, ys, zs, leaks = [], [], [], []
    matrices = []

    for _ in range(steps):
        x, y, z, l = get_bloch_vector(psi)
        xs.append(x); ys.append(y); zs.append(z); leaks.append(l)
        matrices.append(np.abs(U_cum))
        psi = np.dot(U_step, psi)
        psi = normalize_state(psi)
        U_cum = np.dot(U_step, U_cum)

    # --- 2. Setup Figure ---
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'heatmap'}]],
        subplot_titles=("Bloch Sphere (State)", "Rotation Matrix Magnitude |U^n|"),
        column_widths=[0.6, 0.4]
    )

    # Static Background
    phi = np.linspace(0, 2*np.pi, 30)
    theta = np.linspace(0, np.pi, 30)
    sphere_x = np.outer(np.cos(phi), np.sin(theta))
    sphere_y = np.outer(np.sin(phi), np.sin(theta))
    sphere_z = np.outer(np.ones(30), np.cos(theta))
    fig.add_trace(go.Surface(x=sphere_x, y=sphere_y, z=sphere_z, opacity=0.1, showscale=False, colorscale='Greys'), row=1, col=1)

    # Dynamic Traces (Initialized)
    fig.add_trace(go.Scatter3d(x=[0, xs[0]], y=[0, ys[0]], z=[0, zs[0]], mode='lines', line=dict(color='black', width=5)), row=1, col=1)
    fig.add_trace(go.Scatter3d(x=xs[:1], y=ys[:1], z=zs[:1], mode='lines', line=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter3d(x=[xs[0]], y=[ys[0]], z=[zs[0]], mode='markers', marker=dict(size=5, color='red')), row=1, col=1)
    fig.add_trace(go.Heatmap(z=matrices[0], colorscale='Viridis', zmin=0, zmax=1), row=1, col=2)

    fig.update_layout(title="Rendering Video...", height=600, scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='cube'))

    # --- 3. Render Frames ---
    print(f"2. Rendering {len(xs)} frames to images (this takes about 1-2 minutes)...")
    if not os.path.exists('frames'):
        os.makedirs('frames')

    for k in range(len(xs)):
        # Manually update data for the static export
        # Trace 1: Vector
        fig.data[1].x = [0, xs[k]]; fig.data[1].y = [0, ys[k]]; fig.data[1].z = [0, zs[k]]
        # Trace 2: Trail
        fig.data[2].x = xs[:k+1]; fig.data[2].y = ys[:k+1]; fig.data[2].z = zs[:k+1]
        # Trace 3: Tip
        fig.data[3].x = [xs[k]]; fig.data[3].y = [ys[k]]; fig.data[3].z = [zs[k]]
        fig.data[3].marker.color = leaks[k]
        # Trace 4: Heatmap
        fig.data[4].z = matrices[k]

        # Save frame
        # Using engine='kaleido' explicitly
        fig.write_image(f"frames/frame_{k:03d}.png", engine="kaleido", width=1000, height=600)

    # --- 4. Stitch Video ---
    print("3. Stitching frames with FFmpeg...")
    !ffmpeg -y -framerate 10 -i frames/frame_%03d.png -c:v libx264 -pix_fmt yuv420p {filename} > /dev/null 2>&1

    print(f"4. Done! Downloading {filename}...")
    files.download(filename)

# Run the export with default parameters
export_sfq_video(kick_angle=0.15, leakage_lambda=0.2, steps=50)

Checking environment for video export...
Kaleido is installed (version unknown).
Performing hard reset of Plotly modules...
Successfully purged 105 modules.
1. Initializing Simulation (Angle=0.15, Lambda=0.2)...
2. Rendering 50 frames to images (this takes about 1-2 minutes)...


/tmp/ipython-input-1564813356.py:104: DeprecationWarning:


Support for the 'engine' argument is deprecated and will be removed after September 2025.
Kaleido will be the only supported engine at that time.


/tmp/ipython-input-1564813356.py:104: DeprecationWarning:


Support for the 'engine' argument is deprecated and will be removed after September 2025.
Kaleido will be the only supported engine at that time.


/tmp/ipython-input-1564813356.py:104: DeprecationWarning:


Support for the 'engine' argument is deprecated and will be removed after September 2025.
Kaleido will be the only supported engine at that time.


/tmp/ipython-input-1564813356.py:104: DeprecationWarning:


Support for the 'engine' argument is deprecated and will be removed after September 2025.
Kaleido will be the only supported engine at that time.


/tmp/ipython-input-1564813356.py:104: DeprecationWarning:


Support for the 'engine' argument is deprecated and will be removed after September 2025.
Kaleido will be th

3. Stitching frames with FFmpeg...
4. Done! Downloading sfq_video.mp4...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>